In [20]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions

In [68]:
using MacroTools

In [87]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    
    x_prev = x_prior
    
    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P) where { 
            dependencies = WithInboundFunctionalDependencies((2, ), (nothing, )) 
        }
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [88]:
P = 1.0
n = 5

data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [89]:
function inference(data, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, 1.0, P);

    x_buffer  = buffer(Marginal, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, x_buffer)
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return getvalues(x_buffer)
end

inference (generic function with 1 method)

In [90]:
inference(data, x0_prior, P)

LoadError: MethodError: no method matching findfirst(::Int64, ::Tuple{Int64})
[0mClosest candidates are:
[0m  findfirst([91m::Function[39m, ::Any) at array.jl:1898
[0m  findfirst(::Any) at array.jl:1814

In [49]:
x0_prior = NormalMeanVariance(0.0, 10000.0)
@benchmark res = inference($data, $x0_prior, $P)

BenchmarkTools.Trial: 
  memory estimate:  129.59 KiB
  allocs estimate:  2142
  --------------
  minimum time:     146.537 μs (0.00% GC)
  median time:      177.727 μs (0.00% GC)
  mean time:        202.285 μs (8.60% GC)
  maximum time:     7.810 ms (95.17% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [6]:
x0_prior = NormalMeanVariance(0.0, 10000.0)
@benchmark res = inference($data, $x0_prior, $P)

BenchmarkTools.Trial: 
  memory estimate:  12.28 MiB
  allocs estimate:  206592
  --------------
  minimum time:     14.879 ms (0.00% GC)
  median time:      16.607 ms (0.00% GC)
  mean time:        18.578 ms (11.87% GC)
  maximum time:     29.016 ms (39.59% GC)
  --------------
  samples:          269
  evals/sample:     1

In [5]:
x0_prior = NormalMeanVariance(0.0, 10000.0)
@benchmark res = inference($data, $x0_prior, $P)

BenchmarkTools.Trial: 
  memory estimate:  12.28 MiB
  allocs estimate:  206592
  --------------
  minimum time:     14.691 ms (0.00% GC)
  median time:      16.059 ms (0.00% GC)
  mean time:        18.036 ms (12.46% GC)
  maximum time:     26.821 ms (38.14% GC)
  --------------
  samples:          277
  evals/sample:     1

In [14]:
inference(data, x0_prior, P)

500-element Vector{Marginal}:
 Marginal(NormalMeanVariance{Float64}(μ=0.9887735574697237, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=1.988773557469724, v=0.0019999996000000823))
 Marginal(NormalMeanVariance{Float64}(μ=2.988773557469723, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=3.988773557469723, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=4.988773557469723, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=5.988773557469724, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=6.988773557469723, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=7.988773557469725, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=8.988773557469722, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=9.988773557469722, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=10.988773557469722, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}